<a href="https://colab.research.google.com/github/Vanpersact/hello-world/blob/main/BarChartRace_Manifestacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Bar Chart Race — Manifestações por período (diário/mensal)

Gera um **ranking animado** a partir de uma base em formato longo com:

- `data` (YYYY-MM-DD ou YYYY-MM)
- `categoria` (ex.: Linha, Tipo)
- `valor` (quantidade no período)

Ajuste as variáveis na seção **Configuração** e rode as células.


In [24]:

# Bibliotecas
import os, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter

pd.set_option("display.max_rows", 10)



## 1) Configuração


In [25]:

# >>>>>>> edite aqui <<<<<<<
ARQUIVO_DADOS = "Manifestacoes.csv"   # ex.: "manifestacoes.csv"  (deixe None para usar um exemplo sintético)
FREQ = "D"            # "D" = diário | "M" = mensal
TOP_N = 20            # Top N do ranking
STEPS_PER_PERIOD = 10 # suavização (maior = mais liso e mais pesado)
FPS = 20              # frames por segundo
SAIDA = "bar_chart_race_manifestacoes.gif"  # .gif (Pillow) ou .mp4 (se FFMPEG disponível)

# Se suas colunas tiverem outros nomes, ajuste aqui:
COL_DATA = "DIA"
COL_CATEGORIA = "MOTIVO"
COL_VALOR = "QTDE"



## 2) Carregamento da base


In [26]:

def carregar_base(arquivo):
    import pandas as pd

    # 1) Leitura (autodetecta separador de CSV)
    if arquivo.lower().endswith(".csv"):
        df = pd.read_csv(arquivo, sep=None, engine="python", encoding="utf-8-sig")
    else:
        df = pd.read_excel(arquivo)

    # 2) Normaliza nomes de colunas para minúsculas e sem espaços
    df.rename(columns={c: c.strip().lower() for c in df.columns}, inplace=True)

    # 3) Mapeia sinônimos -> alvos ('data', 'categoria', 'valor')
    wanted = {
        "data":      ["data", "dia", "mes_ano", "mesano", "mes", "date"],
        "categoria": ["categoria", "linha", "motivo", "tipo", "classe"],
        "valor":     ["valor", "qtd", "qtde", "quantidade", "manifestacoes", "manifestações", "volume"],
    }

    for target, candidates in wanted.items():
        if target not in df.columns:
            for cand in candidates:
                if cand in df.columns:
                    df.rename(columns={cand: target}, inplace=True)
                    break

    # 4) Checagem final
    missing = [c for c in ["data", "categoria", "valor"] if c not in df.columns]
    if missing:
        raise ValueError(f"Colunas ausentes: {missing}. Colunas encontradas: {list(df.columns)}")

    # 5) Tipagem/limpeza
    # dayfirst=True cobre datas no padrão brasileiro (dd/mm/aaaa)
    df["data"] = pd.to_datetime(df["data"], errors="coerce", dayfirst=True)
    df = df.dropna(subset=["data", "categoria", "valor"]).copy()

    df["categoria"] = df["categoria"].astype(str).str.strip()
    df["valor"] = pd.to_numeric(df["valor"], errors="coerce").fillna(0).astype(int)

    return df


## 3) Preparação do painel


In [27]:

def preparar_painel(df, freq="D"):
    df = df.copy()
    if freq not in ("D","M"):
        raise ValueError("freq deve ser 'D' (diário) ou 'M' (mensal).")
    if freq == "M":
        df["periodo"] = df[COL_DATA].dt.to_period("M").dt.to_timestamp()
    else:
        df["periodo"] = df[COL_DATA].dt.floor("D")
    agg = (df.groupby(["periodo", COL_CATEGORIA], as_index=False)[COL_VALOR].sum()
             .sort_values(["periodo", COL_VALOR], ascending=[True, False]))
    painel = agg.pivot_table(index="periodo", columns=COL_CATEGORIA, values=COL_VALOR, fill_value=0)
    painel = painel.sort_index()
    return painel, agg

painel, agg = preparar_painel(df, FREQ)
painel.head()


NameError: name 'df' is not defined


## 4) Função do bar chart race


In [ ]:

from matplotlib.cm import get_cmap

def bar_chart_race(painel, top_n=10, steps_per_period=10, fps=20, saida="bar_chart_race.gif", titulo=None):
    painel = painel.copy().astype(float).fillna(0.0)
    periods = painel.index.to_list()
    cats = list(painel.columns)
    cmap = get_cmap("tab20")
    cores = {c: cmap(i % 20) for i, c in enumerate(sorted(cats))}
    frames = []
    for i in range(len(periods)-1):
        a, b = periods[i], periods[i+1]
        va = painel.loc[a]
        vb = painel.loc[b]
        union_cats = sorted(set(va.index) | set(vb.index))
        va = va.reindex(union_cats).fillna(0)
        vb = vb.reindex(union_cats).fillna(0)
        for s in range(steps_per_period):
            t = s / float(steps_per_period)
            v = va*(1-t) + vb*t
            frames.append((a, v))
    frames.append((periods[-1], painel.iloc[-1].reindex(cats).fillna(0)))
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_facecolor("#f8f8f8")
    fig.patch.set_facecolor("#f8f8f8")
    ax.grid(axis="x", zorder=0, linestyle="--", alpha=0.3)
    title_text = ax.text(0.5, 1.05, titulo or "Bar Chart Race", transform=ax.transAxes,
                         ha="center", va="bottom", fontsize=16, weight="bold")
    date_text = ax.text(0.98, 0.95, "", transform=ax.transAxes, ha="right", va="top",
                        fontsize=12, family="monospace")
    def draw_frame(frame_data):
        periodo, valores = frame_data
        ax.clear()
        ax.set_facecolor("#f8f8f8")
        ax.grid(axis="x", zorder=0, linestyle="--", alpha=0.3)
        top = valores.sort_values(ascending=False).head(top_n)[::-1]
        ax.barh(top.index, top.values, color=[cores[c] for c in top.index], zorder=3)
        for cat, val in zip(top.index, top.values):
            ax.text(val, cat, f" {int(val):,}".replace(",", "."), va="center", ha="left", fontsize=11)
        ax.set_xlim(0, max(1, valores.max())*1.15)
        ax.set_xlabel("Quantidade")
        ax.set_ylabel("Categoria")
        title_text.set_text(titulo or "Bar Chart Race")
        date_text.set_text(pd.to_datetime(periodo).strftime("%Y-%m-%d"))
    ani = animation.FuncAnimation(fig, draw_frame, frames=frames, interval=1000/fps, blit=False, repeat=False)
    if saida.lower().endswith(".gif"):
        ani.save(saida, writer=PillowWriter(fps=fps))
    else:
        try:
            ani.save(saida, fps=fps)
        except Exception as e:
            print("FFMPEG não disponível. Salve como .gif ou instale FFMPEG. Erro:", e)
    plt.close(fig)
    return saida



## 5) Gerar o GIF/MP4


In [ ]:

titulo = f"Manifestações — Ranking por {('dia' if FREQ=='D' else 'mês')} (Top {TOP_N})"
arquivo_saida = bar_chart_race(
    painel,
    top_n=TOP_N,
    steps_per_period=STEPS_PER_PERIOD,
    fps=FPS,
    saida=SAIDA,
    titulo=titulo
)
arquivo_saida
